# Generate UNPARAMETERIZED Synthetic Data

In [86]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
import geomstats.backend as gs

os.environ["GEOMSTATS_BACKEND"] = "autograd"

print(os.environ["GEOMSTATS_BACKEND"])

autograd


In [92]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# #this forces the notebook to re-load
# import importlib
# importlib.reload(os.path.join(my28brains_dir, "discrete_surfaces.py"))

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

print(os.environ["GEOMSTATS_BACKEND"])

TypeError: reload() argument must be a module

# Generate Source/Target meshes

## 1. Sphere Mesh

In [78]:
sphere = trimesh.creation.icosphere(subdivisions=3, radius=1.0)

#sphere.show()

## 1 Ellipsoid Mesh

In [79]:
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

In [80]:
print(os.environ["GEOMSTATS_BACKEND"])

autograd


# Define surface space

In [81]:
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)
METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

print(os.environ["GEOMSTATS_BACKEND"])

autograd


In [82]:
n_vertices = sphere.vertices.shape[0]
print(n_vertices)

def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3

    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        geodesics[i_geodesic] = geodesic(times)

    return geodesics

642


In [83]:
print(os.environ["GEOMSTATS_BACKEND"])

autograd


In [84]:
geodesics_sphere_to_ellipsoid()

RuntimeError: Automatic differentiation is not supported with numpy backend. Use autograd or pytorch backend instead.
Change backend via the command export GEOMSTATS_BACKEND=autograd in a terminal.